In [ ]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import schedule
import time
import pickle



article_links = [] 
titles = []
names = []
dates = [] 
Visited_webpages_page = []
Visited_webpages_authors = []
profiles = []
profile_urls = []
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'}

#profile collection

def scholar_spider():
    '''Function to crawler google scholar'''
    max_pages = 1
    start_pages = 0
    max_article = "0 - 0"
    after_a =  " "
    after_author = " "
    item_url = " "
    count = 0  
    page = 1 
    url = 'y'
    url1 = 'y'
    while page >= max_pages:   
        print('page number is:', page)
        print('max page is:', max_pages)
        
  
        start_pages = (max_article.split()[2])
        url = 'https://scholar.google.co.uk/citations?view_op=view_org&hl=en&org=9117984065169182779' + str("&after_author=") + str(after_author) + str('&astart=') + str(start_pages) #allows url to change as it traverses through each webpage
        
        print('url is', url)
        Visited_webpages_page.append(url)
        time.sleep(random.uniform(10,15))
        source_code = requests.get(url, headers = headers)
        plain_text = source_code.text 
        soup =BeautifulSoup(plain_text) 
        
        #profile links
        
        for link in soup.findAll('a', {'class': 'gs_ai_pho'}):    
            profile = 'https://scholar.google.co.uk' + link.get('href')
            profile_urls.append(profile)
        for link in soup.findAll('button', {'class' : 'gs_btnPR gs_in_ib gs_btn_half gs_btn_lsb gs_btn_srt gsc_pgn_pnx'}):
            after_a = link.get('onclick')
            # breaks while loop when no next page
            if after_a == None:
                max_pages = page + 1
                page = 0
            else:
                after_author = after_a[109:121]
                
                for link in soup.findAll('span', {'class': 'gs_nph gsc_pgn_ppn'}):
                    max_article = link.string
                    max_pages = (max_article.split())[2]
                    max_pages = int(int(max_pages)/10)+1    
                    print(max_pages)
                page += 1
        print('max pages at end', max_pages)
                
        print('pages at end:', page)
    print(access_all())
    
       
        
# article document information

def access_all():
    '''function to extract document information'''
    for p in profile_urls:
        startpage = 0
        item_url1= 'yes'
        item_url2 = 'yes'
        print(p)
        while item_url1 == item_url2:
            item_url1 = p + str('&cstart=') + str(startpage) + str('&pagesize=100') 
            Visited_webpages_authors.append(item_url1)  
            source_code = requests.get(item_url1, headers=headers)
            plain_text = source_code.text
            soup =BeautifulSoup(plain_text)
            #scrapes links and titles
            for article in soup.findAll('a', {'class': 'gsc_a_at'}):
                article_links.append('https://scholar.google.co.uk' + article.get('data-href'))
                titles.append(article.string)
            #scrapes dates
            for link in soup.findAll('span', {'class':'gsc_a_h gsc_a_hc gs_ibl'}):
                date = (link.string)
                dates.append(date)
                #scrapes authors
                for link in soup.findAll('div', {'id': 'gsc_prf_in'}):
                    names.append(link.string)
            for link in soup.findAll('span', {'id': 'gsc_a_nn'}):
                article_number = link.string
                print(article_number)
                startpage = int(article_number.split("–",1)[1])
                item_url2 = item_url1
                
   
        
    
    


#schedule once a week

schedule.every().thursday.at("17:02").do(scholar_spider)


#must remain open to stop shell ending
while True:
    #turns numbers into strings
    titles1 = list( map(str, titles) )
    dates1 = list( map(str, dates) )
    titles = titles1
    dates = dates1
    
#main document list    
    aa = [x + str(" ") + c + str(" ") + z for x, c, z in zip(titles, dates, names)]
    
    
    Dic1 = dict(list(enumerate(article_links)))
    Dic2 = dict(list(enumerate(aa)))
    Dic3 = dict(list(enumerate(titles)))
    Dic4 = dict(list(enumerate(dates)))
    Dic5 = dict(list(enumerate(names)))
    
    
#save dictionaries to a file 
    with open('aa', 'wb') as fp:
        pickle.dump(aa, fp)

    with open('Dic5', 'wb') as fp:
        pickle.dump(Dic5, fp)    


    with open('Dic3', 'wb') as fp:
        pickle.dump(Dic3, fp)    
    
    with open('Dic4', 'wb') as fp:
        pickle.dump(Dic4, fp)
    
    with open('Dic2', 'wb') as fp:
        pickle.dump(Dic2, fp)

    with open('Dic1', 'wb') as fp:
        pickle.dump(Dic1, fp)
    
    schedule.run_pending()

